# Практическая работа №2
----
**Выполнил: Ковалев А.И.**

**Группа: ПИ19-3**

Структура проекта: **src/** - директория с кодом, **resources/** - директория с исходными данными (.xlsx, .sql), **university_life.db** - SQLite бд результат практической, **main.py** - entrypoint для запуска проекта


_Ниже представлен листинг кода:_

### src/models.py

In [5]:
from sqlalchemy.ext.declarative import as_declarative, declared_attr
from sqlalchemy import (Column, Integer, String, DateTime, ForeignKey, CheckConstraint, Float)

CASCADE = "CASCADE"
SHORT_VARCHAR = 80


@as_declarative()
class Base(object):
    id = Column(Integer(), primary_key=True, autoincrement=True)

    def _repr(self, **fields) -> str:
        field_strings = []
        for key, field in fields.items():
            field_strings.append(f'{key}={field!r}')

        return f"<{self.__class__.__name__} {field_strings}>"

    @declared_attr
    def __tablename__(self):
        return self.__name__.lower()

    def __repr__(self) -> str:
        return self._repr(id=self.id)

    __str__ = __repr__


class University(Base):
    name = Column(String(SHORT_VARCHAR), nullable=False)
    rating = Column(Integer())
    city = Column(String(SHORT_VARCHAR), nullable=False)

    def __repr__(self):
        return self._repr(name=self.name, rating=self.rating, city=self.city)


class Student(Base):
    first_name = Column(String(SHORT_VARCHAR), nullable=False)
    last_name = Column(String(SHORT_VARCHAR), nullable=False)
    stipend = Column(Float(), CheckConstraint("stipend >= 0.00", name="stipend_is_positive"))
    course = Column(Integer(), nullable=False)
    birthdate = Column(DateTime())
    university_id = Column(Integer(), ForeignKey("university.id", ondelete=CASCADE))

    def __repr__(self):
        return self._repr(first_name=self.first_name,
                          last_name=self.last_name,
                          stipend=self.stipend,
                          course=self.course,
                          birthdate=self.birthdate,
                          university_id=self.university_id)


class Subject(Base):
    name = Column(String(SHORT_VARCHAR), nullable=False)
    hour = Column(Integer(), nullable=False)
    semester = Column(Integer(), nullable=False)

    def __repr__(self):
        return self._repr(name=self.name, hour=self.hour, semester=self.semester)


class ExamMark(Base):
    student_id = Column(Integer(), ForeignKey("student.id", ondelete=CASCADE))
    subject_id = Column(Integer(), ForeignKey("subject.id", ondelete=CASCADE))
    mark = Column(Integer())
    date = Column(DateTime(), nullable=False)

    def __repr__(self):
        return self._repr(student_id=self.student_id,
                          subject_id=self.subject_id,
                          mark=self.mark,
                          date=self.date)


class ActivityLecturer(Base):
    first_name = Column(String(SHORT_VARCHAR), nullable=False)
    last_name = Column(String(SHORT_VARCHAR), nullable=False)
    city = Column(String(SHORT_VARCHAR))
    university_id = Column(Integer(), ForeignKey("university.id", ondelete=CASCADE))

    def __repr__(self):
        return self._repr(first_name=self.first_name,
                          last_name=self.last_name,
                          city=self.city,
                          university_id=self.university_id)


class SubjectLecturer(Base):
    lecturer_id = Column(Integer(), ForeignKey("activitylecturer.id", ondelete=CASCADE))
    subject_id = Column(Integer, ForeignKey("subject.id", ondelete=CASCADE))

    def __repr__(self):
        return self._repr(lecturer_id=self.lecturer_id, subject_id=self.subject_id)


### src/utils.py

In [4]:
class SingletonMeta(type):
    _instances = {}

    def __call__(cls, *args, **kwargs):
        if cls not in cls._instances:
            instance = super().__call__(*args, **kwargs)
            cls._instances[cls] = instance
        return cls._instances[cls]


### src/database.py

In [6]:
from sqlalchemy import create_engine
from sqlalchemy.orm.session import sessionmaker, Session
from src.models import Base, University, Student, Subject, ExamMark, ActivityLecturer, SubjectLecturer
from src.utils import SingletonMeta

from datetime import datetime

DATABASE = "sqlite"
DB_NAME = "university_life.db"


class DataBase(metaclass=SingletonMeta):

    def __init__(self):
        self.engine = create_engine(f"{DATABASE}:///{DB_NAME}")
        self.session: Session = sessionmaker(bind=self.engine)()

    def _create_all_tables(self):
        Base.metadata.create_all(self.engine)

    def _remove_all_tables(self):
        Base.metadata.drop_all(self.engine)

    def select_university(self, **kwargs):
        return self.session.query(University).filter_by(**kwargs).all()

    def create_university(self, id_: int = None, name: str = None, rating: int = None, city: str = None):
        self._create(University, id=id_, name=name, rating=rating, city=city)

    def update_university(self, id_: int, name: str = None, rating: int = None, city: str = None):
        updatable = {
            University.name: name,
            University.rating: rating,
            University.city: city
        }
        self._update(University, id_, updatable)

    def delete_university(self, id_: int):
        self.session.query(University).filter_by(id=id_).delete()

    def select_student(self, **kwargs):

        return self.session.query(Student).filter_by(**kwargs).all()

    def create_student(self, id_: int = None, first_name: str = None, last_name: str = None, stipend: float = None,
                       course: int = None, birthdate: datetime = None, university_id: int = None):
        self._create(Student, id=id_, first_name=first_name, last_name=last_name, stipend=stipend,
                     course=course, birthdate=birthdate, university_id=university_id)

    def update_student(self, id_: int, first_name: str = None, last_name: str = None, stipend: float = None,
                       course: int = None, birthdate: datetime = None, university_id: int = None):
        updatable = {
            Student.first_name: first_name,
            Student.last_name: last_name,
            Student.stipend: stipend,
            Student.course: course,
            Student.birthdate: birthdate,
            Student.university_id: university_id,
        }
        self._update(Student, id_, updatable)

    def delete_student(self, id_: int):
        self.session.query(Student).filter_by(id=id_).delete()

    def select_subject(self, **kwargs):
        return self.session.query(Subject).filter_by(**kwargs).all()

    def create_subject(self, id_: int = None, name: str = None, hour: int = None, semester: int = None):
        self._create(Subject, id=id_, name=name, hour=hour, semester=semester)

    def update_subject(self, id_: int, name: str = None, hour: int = None, semester: int = None):
        updatable = {
            Subject.name: name,
            Subject.hour: hour,
            Subject.semester: semester
        }
        self._update(Subject, id_, updatable)

    def delete_subject(self, id_: int):
        self.session.query(Subject).filter_by(id=id_).delete()

    def select_exam_mark(self, **kwargs):
        return self.session.query(ExamMark).filter_by(**kwargs).all()

    def create_exam_mark(self, id_: int = None, student_id: int = None, subject_id: int = None,
                         mark: int = None, date: datetime = None):
        self._create(ExamMark, id=id_, student_id=student_id, subject_id=subject_id, mark=mark, date=date)

    def update_exam_mark(self, id_: int, student_id: int = None, subject_id: int = None,
                         mark: int = None, date: datetime = None):
        updatable = {
            ExamMark.student_id: student_id,
            ExamMark.subject_id: subject_id,
            ExamMark.mark: mark,
            ExamMark.date: date
        }
        self._update(ExamMark, id_, updatable)

    def delete_exam_mark(self, id_: int):
        self.session.query(ExamMark).filter_by(id=id_).delete()

    def select_activity_lecturer(self, **kwargs):
        return self.session.query(ActivityLecturer).filter_by(**kwargs).all()

    def create_activity_lecturer(self, id_: int = None, first_name: str = None, last_name: str = None,
                                 city: str = None, university_id: int = None):
        self._create(ActivityLecturer, id=id_, first_name=first_name,
                     last_name=last_name, city=city, university_id=university_id)

    def update_activity_lecturer(self, id_: int = None, first_name: str = None, last_name: str = None,
                                 city: str = None, university_id: int = None):
        updatable = {
            ActivityLecturer.first_name: first_name,
            ActivityLecturer.last_name: last_name,
            ActivityLecturer.city: city,
            ActivityLecturer.university_id: university_id
        }
        self._update(ActivityLecturer, id_, updatable)

    def delete_activity_lecturer(self, id_: int):
        self.session.query(ActivityLecturer).filter_by(id=id_).delete()

    def select_subject_lecturer(self, **kwargs):
        return self.session.query(SubjectLecturer).filter_by(**kwargs).all()

    def create_subject_lecturer(self, id_: int = None, lecturer_id: int = None, subject_id: int = None):
        self._create(SubjectLecturer, id=id_, lecturer_id=lecturer_id, subject_id=subject_id)

    def update_subject_lecturer(self, id_: int = None, lecturer_id: int = None, subject_id: int = None):
        updatable = {
            SubjectLecturer.lecturer_id: lecturer_id,
            SubjectLecturer.subject_id: subject_id,
        }
        self._update(SubjectLecturer, id_, updatable)

    def delete_subject_lecturer(self, id_: int):
        self.session.query(SubjectLecturer).filter_by(id=id_).delete()

    def _create(self, model, **kwargs):
        obj = model(**kwargs)
        self._save(obj)

    def _save(self, obj):
        self.session.add(obj)
        self.session.commit()

    def _update(self, model, id_, updatable):
        updatable = {k: v for k, v in updatable.items() if v}
        self.session.query(model).filter_by(id=id_).update(updatable)


### main.py

In [7]:
import pandas as pd
import numpy as np
from src.database import DataBase
import pprint


def _int(x):
    if isinstance(x, np.integer):
        return int(x)
    return x


def _float(x):
    if isinstance(x, np.floating):
        return float(x)
    return x


def create():
    university: pd.DataFrame = pd.read_excel("./resources/educational_activities_university.xlsx")
    subject: pd.DataFrame = pd.read_excel("./resources/educational_activities_subject.xlsx")
    student: pd.DataFrame = pd.read_excel("./resources/educational_activities_student.xlsx")
    exam_mark: pd.DataFrame = pd.read_excel("./resources/educational_activities_exam_marks.xlsx")
    activity_lecturer: pd.DataFrame = pd.read_excel("./resources/educational_activities_lecturer.xlsx")
    subject_lecturer: pd.DataFrame = pd.read_excel("./resources/educational_activities_subj_lect.xlsx")

    university.rename(columns={"UNIV_ID": "id", "UNIV_NAME": "name",
                               "RATING": "rating", "CITY": "city"}, inplace=True)
    subject.rename(columns={"SUBJ_ID": "id", "SUBJ_NAME": "name",
                            "HOUR": "hour", "SEMESTER": "semester"}, inplace=True)
    student.rename(columns={"STUDENT_ID": "id", "SURNAME": "last_name",
                            "NAME": "first_name", "STIPEND": "stipend",
                            "KURS": "course", "CITY": "city",
                            "BIRTHDAY": "birthdate", "UNIV_ID": "university_id"}, inplace=True)
    exam_mark.rename(columns={"EXAM_ID": "id",
                              "STUDENT_ID": "student_id",
                              "SUBJ_ID": "subject_id",
                              "MARK": "mark",
                              "EXAM_DATE": "date"}, inplace=True)
    activity_lecturer.rename(columns={"LECTURER_ID": "id", "SURNAME": "first_name",
                                      "NAME": "last_name", "CITY": "city",
                                      "UNIV_ID": "university_id"}, inplace=True)
    subject_lecturer.rename(columns={"LECTURER_ID": "lecturer_id", "SUBJ_ID": "subject_id"}, inplace=True)

    db = DataBase()
    for i in university.index:
        db.create_university(id_=_int(university.loc[i, "id"]),
                             name=university.loc[i, "name"],
                             rating=_int(university.loc[i, "rating"]),
                             city=university.loc[i, "city"])

    for i in subject.index:
        db.create_subject(id_=_int(subject.loc[i, "id"]),
                          name=subject.loc[i, "name"],
                          hour=_int(subject.loc[i, "hour"]),
                          semester=_int(subject.loc[i, "semester"]))

    student["birthdate"] = pd.to_datetime(student["birthdate"])
    for i in student.index:
        db.create_student(id_=_int(student.loc[i, "id"]),
                          first_name=student.loc[i, "first_name"],
                          last_name=student.loc[i, "last_name"],
                          stipend=_float(student.loc[i, "stipend"]),
                          course=_int(student.loc[i, "course"]),
                          birthdate=birthdate if (birthdate := student.loc[i, "birthdate"]) is not pd.NaT else None,
                          university_id=_int(student.loc[i, "university_id"]))

    exam_mark["date"] = pd.to_datetime(exam_mark["date"])
    for i in exam_mark.index:
        db.create_exam_mark(id_=_int(exam_mark.loc[i, "id"]),
                            student_id=_int(exam_mark.loc[i, "student_id"]),
                            subject_id=_int(exam_mark.loc[i, "subject_id"]),
                            mark=_int(exam_mark.loc[i, "mark"]),
                            date=date if (date := exam_mark.loc[i, "date"]) is not pd.NaT else None)

    for i in activity_lecturer.index:
        db.create_activity_lecturer(id_=_int(activity_lecturer.loc[i, "id"]),
                                    first_name=activity_lecturer.loc[i, "first_name"],
                                    last_name=activity_lecturer.loc[i, "last_name"],
                                    city=activity_lecturer.loc[i, "city"],
                                    university_id=_int(activity_lecturer.loc[i, "university_id"]))

    for i in subject_lecturer.index:
        db.create_subject_lecturer(lecturer_id=_int(subject_lecturer.loc[i, "lecturer_id"]),
                                   subject_id=_int(subject_lecturer.loc[i, "subject_id"]))


def show():
    db = DataBase()
    pprint.pprint(db.select_student())
    pprint.pprint(db.select_subject_lecturer())
    pprint.pprint(db.select_subject())
    pprint.pprint(db.select_university())
    pprint.pprint(db.select_exam_mark())
    pprint.pprint(db.select_activity_lecturer())


if __name__ == '__main__':
    DataBase()._remove_all_tables()  # удалит все таблицы
    DataBase()._create_all_tables()  # создаст все таблицы заново
    create()  # наполнение таблиц данными
    show()  # отчет


[<Student ["first_name='Иван'", "last_name='Иванов'", 'stipend=150.0', 'course=1', 'birthdate=datetime.datetime(1982, 12, 3, 0, 0)', 'university_id=10']>,
 <Student ["first_name='Петр'", "last_name='Петров'", 'stipend=200.0', 'course=3', 'birthdate=datetime.datetime(1980, 12, 1, 0, 0)', 'university_id=10']>,
 <Student ["first_name='Вадим'", "last_name='Сидоров'", 'stipend=150.0', 'course=4', 'birthdate=datetime.datetime(1979, 6, 7, 0, 0)', 'university_id=22']>,
 <Student ["first_name='Борис'", "last_name='Кузнецов'", 'stipend=0.0', 'course=2', 'birthdate=datetime.datetime(1981, 12, 8, 0, 0)', 'university_id=10']>,
 <Student ["first_name='Ольга'", "last_name='Зайцева'", 'stipend=250.0', 'course=2', 'birthdate=datetime.datetime(1981, 5, 1, 0, 0)', 'university_id=10']>,
 <Student ["first_name='Павел'", "last_name='Котов'", 'stipend=150.0', 'course=5', 'birthdate=None', 'university_id=14']>,
 <Student ["first_name='Вадим'", "last_name='Белкин'", 'stipend=250.0', 'course=5', 'birthdate=date